In [1]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [6]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [7]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [8]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [10]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [11]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [12]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [15]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-4.14056564e-03  1.08433235e-03  2.02269177e-03 ...  1.20480789e-03
    1.52766914e-03 -5.67873335e-03]
  [-4.31528222e-03 -7.91878672e-04 -7.19541777e-03 ... -5.74674131e-03
    3.35317338e-03 -3.50675476e-03]
  [-1.50293543e-03  3.53602972e-03  2.84383353e-03 ... -7.97469914e-03
    3.97134863e-04 -8.99592601e-03]
  ...
  [ 2.76337331e-03  8.78215116e-03  1.04024522e-02 ... -1.50619063e-03
    1.21639529e-02 -6.10082783e-03]
  [ 4.76020761e-03  1.19636208e-02  6.74887095e-04 ...  6.36867248e-04
    1.34632234e-02 -3.82454251e-03]
  [ 2.36404059e-03  5.46256918e-03  3.35026096e-04 ...  9.52087459e-04
    1.23441685e-02 -8.18961672e-03]]

 [[ 6.06970512e-04  2.39540217e-03  1.11724576e-03 ... -2.21652398e-03
   -2.10186630e-03  3.92297469e-03]
  [ 1.31679780e-03  4.53987578e-03  8.92355572e-04 ...  2.82555539e-03
    1.08442083e-02  2.13491335e-03]
  [ 8.74141522e-04  4.29054722e-03 -1.10707479e-03 ...  8.68508033e-03
    7.37642404e-03  1.30316708e-04]
  ...
  [-6.857

In [16]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00414057  0.00108433  0.00202269 ...  0.00120481  0.00152767
  -0.00567873]
 [-0.00431528 -0.00079188 -0.00719542 ... -0.00574674  0.00335317
  -0.00350675]
 [-0.00150294  0.00353603  0.00284383 ... -0.0079747   0.00039713
  -0.00899593]
 ...
 [ 0.00276337  0.00878215  0.01040245 ... -0.00150619  0.01216395
  -0.00610083]
 [ 0.00476021  0.01196362  0.00067489 ...  0.00063687  0.01346322
  -0.00382454]
 [ 0.00236404  0.00546257  0.00033503 ...  0.00095209  0.01234417
  -0.00818962]], shape=(100, 65), dtype=float32)


In [17]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-4.1405656e-03  1.0843324e-03  2.0226918e-03  1.0953355e-03
  3.0071463e-03 -2.3158445e-04  6.1822450e-03  1.1370194e-03
 -4.6029389e-03 -4.3508261e-03 -7.6271559e-04 -1.8469797e-03
 -3.4441021e-03 -9.0333592e-04  2.6899781e-03  6.7069079e-05
  7.0362585e-05 -1.9395959e-03 -7.2775036e-04  2.2960538e-03
 -2.0424107e-03  5.5257371e-04 -2.6483568e-03 -4.5378576e-05
  9.5729937e-04 -4.5932196e-03  4.5129107e-03  1.6334774e-03
  2.5232555e-05 -4.7260909e-03 -4.7311340e-03  6.7850109e-03
 -5.9521543e-03  3.6394615e-03  4.2462457e-04 -2.0341312e-03
  1.6451036e-03  2.1803160e-03 -4.8535992e-04 -4.3144077e-03
 -2.8610763e-03  2.4315470e-04  5.3949503e-04 -2.5805621e-04
 -1.3344134e-03 -2.1602651e-03  1.1488923e-03 -2.1012931e-03
  1.5409433e-03 -4.7679464e-03 -3.7406192e-03  1.5120098e-03
 -7.1631400e-03  1.9905784e-03  1.5653864e-03  5.5053266e-04
 -1.8605326e-03 -5.0201640e-03 -2.1788559e-03  2.4241309e-03
  3.3278117e-04 -7.4643269e-04  1.2048079e-03  1.5276691e-03
 -5.678733

In [18]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

".PHFq'JL$R3H,aVq$wQ-Oz&E,frsdkZrDGeSrVJWWYtkn&UcP\n!\nrCCclZWcbpNugmT3Ncpl3Jdz'iwNbVH3rJBC KhC:;ocws!X"

In [19]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [20]:
model.compile(optimizer='adam', loss=loss)

In [21]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [22]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 17s 65ms/step - loss: 2.5989
Epoch 2/50
172/172 [==============================] - 12s 62ms/step - loss: 1.8901
Epoch 3/50
172/172 [==============================] - 12s 63ms/step - loss: 1.6444
Epoch 4/50
172/172 [==============================] - 14s 68ms/step - loss: 1.5140
Epoch 5/50
172/172 [==============================] - 13s 66ms/step - loss: 1.4335
Epoch 6/50
172/172 [==============================] - 13s 66ms/step - loss: 1.3771
Epoch 7/50
172/172 [==============================] - 13s 66ms/step - loss: 1.3328
Epoch 8/50
172/172 [==============================] - 13s 68ms/step - loss: 1.2938
Epoch 9/50
172/172 [==============================] - 13s 69ms/step - loss: 1.2612
Epoch 10/50
172/172 [==============================] - 13s 69ms/step - loss: 1.2276
Epoch 11/50
172/172 [==============================] - 14s 71ms/step - loss: 1.1949
Epoch 12/50
172/172 [==============================] - 14s 71ms/step - loss: 1.1622
E

In [23]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [24]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [26]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension

      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [28]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: sun
sung frame of Coriolanus music grave
As e'er my mistress, one hath pattern gates i' the tows
To find the all he fell a lift
Of mortal tred therefore?

DUKE VINCENTIO:
I know not where before thou diest to-morrow.

LORD FITZWATER:
How fares the prince?

Messenger:
The will shall be his coming hither to command
At up, a beggar bennd colouring;
back, that he is band for loving praise
Live in thy touch, whose coward as to-joint widows,
Dismiss the THIO:
Spiece! we'll die.

ISABELLA:
I do, my lord; your brother, tell me, hated the corse
That joy had worn it out.

DUKE VINCENTIO:
Sended to the rotten years another rock,
That bounds be a brief.

PAGE:
There is no matter; I will lie with thee here:
O, that a man entake his late kingdom fear'd
And frantis mean to tast when I parter.

GLOUCESTER:
Have 
